In [236]:
import requests
from skimage import io
import cv2
import json
import os
import pprint

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

os.sys.path

['',
 '/home/kaijuan/anaconda3/lib/python36.zip',
 '/home/kaijuan/anaconda3/lib/python3.6',
 '/home/kaijuan/anaconda3/lib/python3.6/lib-dynload',
 '/home/kaijuan/anaconda3/lib/python3.6/site-packages',
 '/home/kaijuan/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kaijuan/.ipython']

In [291]:
# image feature generation using azure img api and open cv generation
cv_key = open('azure_cfg.txt','r')
# subscription_key = cv_key.read()
subscription_key_img = '2c5991ae4f584a9b8acd1152cc48da4c'

vision_base_url="https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyse_url=vision_base_url + 'analyze?'

In [292]:
def extract_high_level_features(image_path):
    headers={'Ocp-Apim-Subscription-Key':subscription_key_img,
             'Content-Type': 'application/json'}
    params = {'visualfeatures':'Adult, Categories, Description, Color, Faces, ImageType, Tags'}
    data = {'url':image_path}

    response = requests.post(vision_analyse_url, headers=headers, params=params, json=data)
    response.raise_for_status()
    data_back = response.json()
    
    features = {
        "clip_art_type"                      : data_back["imageType"]["clipArtType"],
        "line_drawing_type"                  : data_back["imageType"]["lineDrawingType"],
        "is_black_and_white"                 : int(data_back["color"]["isBwImg"]),
        "is_adult_content"                   : int(data_back["adult"]["isAdultContent"]),
        "adult_score"                        : data_back["adult"]["adultScore"],
        "is_racy"                            : int(data_back["adult"]["isRacyContent"]),
        "racy_score"                         : data_back["adult"]["racyScore"],
        "has_faces"                          : int(len(data_back["faces"])),
        "num_faces"                          : len(data_back["faces"]),
        "is_dominant_color_background_black" : int(data_back["color"]["dominantColorBackground"] == "Black"),
        "is_dominant_color_foreground_black" : int(data_back["color"]["dominantColorForeground"] == "Black")
    }
    
#     return features
    return list(features.values())

In [293]:
width, height = 128, 64
descriptor = cv2.HOGDescriptor(_winSize = (width,height),
                               _blockSize = (16,16),
                               _blockStride = (8,8),
                               _cellSize = (8,8),
                               _nbins = 9)

# compute HOG features for an image
def extract_low_level_features(image_path, descriptor=descriptor):
    image = io.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width,height))

    hog = descriptor.compute(image)
    hog = hog.flatten().tolist()
    return hog

In [294]:
# conduct low- and high-level feature extraction for images in a directory
def extract_all_features(image_url_list):
    features = []
    for path in range(len(image_url_list)):
#         print(path)
        high_level_features = extract_high_level_features(image_url_list[path])
#         low_level_features = extract_low_level_features(image_url_list[path])
        
        features.append(high_level_features)
    
#         features.append({**high_level_features})
    return features

In [295]:
def read_img_url(jsonFile):
    data = json.load(open(jsonFile))
    server_img_url_list=[]
    for j in range(0,400):
        server_img_url = 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/'+ data[j]['image']
        server_img_url_list.append(server_img_url)
    return server_img_url_list

In [296]:
train_img_url_list = read_img_url('train.json')
train__img_features = extract_all_features(train_img_url_list)


In [337]:
test_img_url_list=read_img_url('test.json')[:30]
print(test_img_url_list)
test_img_features=extract_all_features(test_img_url_list)
# print(test_img_features)

['https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020000.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020001.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020002.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020003.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020004.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020005.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020006.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020007.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020008.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020009.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020010.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_tes

In [338]:
from pandas import DataFrame
from IPython.display import display

# print(train__img_features)
# df = DataFrame(data=train__img_features)
# display(df)

df = DataFrame(data=test_img_features)
display(df)

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0.035585,0,0.025597,0,0,1,1
1,0,0,0,0,0.067857,0,0.084319,0,0,1,0
2,0,0,0,0,0.007775,0,0.009652,0,0,0,0
3,0,0,0,0,0.017246,0,0.014839,0,0,0,1
4,0,0,0,0,0.009939,0,0.013404,0,0,1,0
5,0,0,0,0,0.027363,0,0.030732,0,0,0,0
6,0,0,0,0,0.022667,0,0.039295,0,0,0,0
7,0,0,0,0,0.084678,0,0.145256,0,0,1,0
8,0,0,0,0,0.047701,0,0.063104,0,0,0,0
9,0,0,0,0,0.069645,0,0.043704,0,0,0,0


In [19]:
'''
text feature extration
'''

import nltk
import pprint
import requests
import json
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/kaijuan/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading p

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-da

True

In [339]:
subscription_key_text = 'f7107338c32d4fcb9191477c4227e962'

text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"
language_api_url = text_analytics_base_url + "languages"
sentiment_api_url = text_analytics_base_url + "sentiment"
key_phrase_api_url = text_analytics_base_url + "keyPhrases"



In [340]:

# evaluate text "documents" using the Microsoft Azure Cognitive Services Text Analytics API
def analyze_text(documents):
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key_text}
    
    # query API for language analysis
    response  = requests.post(language_api_url, headers=headers, json=documents)
    languages = response.json()
    
    # query API for sentiment analysis
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    # query API for key phrases extraction
    response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
    key_phrases = response.json()
    
    return (languages, sentiments, key_phrases)


In [341]:
def pos_tag_text(documents):
    tag_counts = []
    
    for doc in documents['documents']:
        text = nltk.word_tokenize(doc['text'])
        tags = nltk.pos_tag(text)
        frequencies = nltk.FreqDist(tag for (word, tag) in tags)
        tag_count={
            'ADJ':0,
            'ADV':0,
            'CONJ':0,
            'NOUN':0,
            'NUM':0,
            'PRON':0,
            'VERB':0
        }
        
        for tag, count in frequencies.most_common():
            tag_count[tag] =count
        tag_counts.append(tag_count)
    return tag_counts


In [302]:
# get the question documents from the training set
def get_train_questions(json_file):
#     labels = json.load(open('train.json'))
    labels = json.load(open(json_file))
    documents = []
    for index, document in enumerate(labels):
        if index < 400:
            documents.append({
                'id': index,
                'text': document['question']
            })
    
    return {'documents': documents}

In [342]:
# get the question documents from the training set
def get_test_questions(json_file):
#     labels = json.load(open('train.json'))
    labels = json.load(open(json_file))
    documents = []
    for index, document in enumerate(labels):
        if index < 30:
            documents.append({
                'id': index,
                'text': document['question']
            })
    
    return {'documents': documents}

In [344]:
# train_documents = get_train_questions('train.json')
# pprint.pprint(train_documents)

test_documents = get_test_questions('test.json')
pprint.pprint(test_documents)

{'documents': [{'id': 0, 'text': 'What is this? And what color is it?'},
               {'id': 1, 'text': 'What is this?'},
               {'id': 2,
                'text': 'Has this oven gotten up to four hundred fifty degrees '
                        'Fahrenheit yet?'},
               {'id': 3, 'text': 'What is this?'},
               {'id': 4, 'text': 'What is this?'},
               {'id': 5, 'text': 'What kind of key is this?'},
               {'id': 6, 'text': 'What does it say on here?'},
               {'id': 7, 'text': 'What is this? '},
               {'id': 8, 'text': 'What is this? What is this? '},
               {'id': 9,
                'text': 'Do these beans look like black beans or pinto beans?'},
               {'id': 10, 'text': 'Whats the date?'},
               {'id': 11, 'text': 'what is this?'},
               {'id': 12, 'text': 'What is this game?'},
               {'id': 13, 'text': 'This piece of paper.'},
               {'id': 14, 'text': 'What brand is thi

In [203]:
def get_tfidf_doc():
    labels = json.load(open('train.json'))
    for ele in labels:
        tfidf_doc = []
        for index, document in enumerate(labels):
            if index < 400:
                tfidf_doc.append(document['question'])
    return tfidf_doc
tfidf_doc = get_tfidf_doc()



["What's the name of this product?",
 'Can you tell me what is in this can please?',
 'Is this enchilada sauce or is this tomatoes?  Thank you.',
 'What is the captcha on this screenshot?',
 'What is this item?',
 "What's this?",
 'What is in this bottle?',
 'This item.',
 'What color do these look?',
 'Surface look clean? Thank you.',
 'Is this.',
 'What is the sodium content of this can of food?',
 'what is this?',
 'What kind of drink is this?',
 'What flavor are these?',
 'What color... what color is this skirt?',
 'How much money is on the table?',
 'What is the name of this product? What is the name of this product?',
 'What is this? ',
 'Can I ask about anything or any information without taking any pictures?',
 'What is this game?',
 'Alright, and what does this label say?',
 'What is in this can?',
 'What is this movie?',
 'Can you see the piece of mail and can you tell me who this mail is for?',
 'What is this a picture of?',
 'What is this?',
 'Am I right when I say that thi

In [216]:
'''
get tfidf
'''
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

tfidf_train = vectorizer.fit_transform(tfidf_doc)
tfidf_train.shape
print(tfidf_train.get_feature_names())
print(tfidf_train.shape)


AttributeError: get_feature_names not found

In [345]:
# languages, sentiments, key_phrases = analyze_text(train_documents)

languages, sentiments, key_phrases = analyze_text(test_documents)


languages, sentiments, key_phrases = languages['documents'], sentiments['documents'], key_phrases['documents']

# tag_counts = pos_tag_text(train_documents)
tag_counts = pos_tag_text(test_documents)
tag_features=[]

for tag in tag_counts:
    temp_list=[]
    for i in tag:
        temp_list.append(tag[i])
    tag_features.append(temp_list)
print(len(tag_features[0]))
    

14


In [353]:
azure_features=[]

for i in range(len(test_documents['documents'])):
    analysis=[]
    analysis.append(sentiments[i]['score'])
    analysis.append(len(key_phrases[i]['keyPhrases']))
    azure_features.append(analysis)
# print(len(azure_features[0]))

train_features=[]
test_features=[]

    
for i in range(len(test_documents['documents'])):

#     train_features.append(azure_features[i]+train__img_features[i]+tag_features[i])
    test_features.append(azure_features[i]+test_img_features[i]+tag_features[i])
    
print(test_features)
print(len(test_features[0]))


[[0.7486321926116943, 1, 0, 0, 0, 0, 0.03558473661541939, 0, 0.0255971048027277, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1], [0.21035191416740417, 0, 0, 0, 0, 0, 0.06785748898983002, 0, 0.08431851118803024, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0.5, 2, 0, 0, 0, 0, 0.007774937432259321, 0, 0.009651836939156055, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0.21035191416740417, 0, 0, 0, 0, 0, 0.01724584586918354, 0, 0.014839480631053448, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0.21035191416740417, 0, 0, 0, 0, 0, 0.009938917122781277, 0, 0.013403681106865406, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0.765828013420105, 1, 0, 0, 0, 0, 0.027362626045942307, 0, 0.030732067301869392, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1], [0.6965221762657166, 0, 0, 0, 0, 0, 0.02266673743724823, 0, 0.03929518535733223, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0.21035191416740417, 0, 0, 0, 0, 0, 0.08467785269021988, 0, 0.145255938

In [359]:
from pandas import DataFrame
import numpy as np
from IPython.display import display

# df = DataFrame(data=train_features)
# df=df.iloc[:,0:26]
# df_train=df.replace(np.nan,0)
# display(df_train)

df_test = DataFrame(data=test_features)
df_test=df_test.iloc[:,0:26]
df_test=df_test.replace(np.nan,0)
display(df_test)
# df.to_csv('train_200.csv',sep='\t',index=False, encoding='utf-8')

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.748632,1,0,0,0,0,0.035585,0,0.025597,0,...,0,0,0,0,2,2,2,1,1.0,1.0
1,0.210352,0,0,0,0,0,0.067857,0,0.084319,0,...,0,0,0,0,1,1,1,1,0.0,0.0
2,0.500000,2,0,0,0,0,0.007775,0,0.009652,0,...,0,0,0,0,2,2,1,1,1.0,1.0
3,0.210352,0,0,0,0,0,0.017246,0,0.014839,0,...,0,0,0,0,1,1,1,1,0.0,0.0
4,0.210352,0,0,0,0,0,0.009939,0,0.013404,0,...,0,0,0,0,1,1,1,1,0.0,0.0
5,0.765828,1,0,0,0,0,0.027363,0,0.030732,0,...,0,0,0,0,2,1,1,1,1.0,1.0
6,0.696522,0,0,0,0,0,0.022667,0,0.039295,0,...,0,0,0,0,1,1,1,1,1.0,1.0
7,0.210352,0,0,0,0,0,0.084678,0,0.145256,0,...,0,0,0,0,1,1,1,1,0.0,0.0
8,0.210352,0,0,0,0,0,0.047701,0,0.063104,0,...,0,0,0,0,2,2,2,2,0.0,0.0
9,0.500000,2,0,0,0,0,0.069645,0,0.043704,0,...,0,0,0,0,3,1,1,1,1.0,1.0


In [325]:
# get the class labels for the training instances
def get_class_labels():
    train_labels = json.load(open('train.json'))
    
    train_classes = [train_labels[label]['answerable'] for label in range(0,400)]
    return train_classes

In [326]:
train_target = get_class_labels()
print('Train target:\n', train_target)

Train target:
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 

In [333]:
'''
binary classification model
'''
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train, train_target, random_state=42)
# pprint.pprint(X_test)

# rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
# kfold=KFold(n_splits=5)
# fold_accuarcy=cross_val_score(rnd_clf,df,train_target,cv=kfold)
# print("CV SCORE for RForest:{.2f}",fold_accuarcy)

# rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
# rnd_clf.fit(X=X_train, y=y_train)

# y_pred_rf = rnd_clf.predict(X_test)
# print(y_pred_rf)

           0   1   2   3   4   5         6   7         8   9  ...   16  17  \
209  0.833592   1   0   0   0   0  0.035652   0  0.020891   0 ...    0   0   
280  0.768412   2   0   0   0   0  0.039585   0  0.035437   0 ...    0   0   
33   0.975901   3   0   0   0   0  0.013949   0  0.066677   0 ...    0   0   
210  0.765828   1   0   0   0   0  0.046437   0  0.076497   0 ...    0   0   
93   0.210352   0   0   0   0   0  0.028021   0  0.031107   0 ...    0   0   
84   0.761298   1   0   0   0   0  0.010262   0  0.014999   0 ...    0   0   
329  0.890254   1   0   0   0   0  0.027984   0  0.018624   0 ...    0   0   
94   0.252677   1   0   0   0   0  0.020042   0  0.026386   0 ...    0   0   
266  0.710597   1   1   0   0   0  0.055924   0  0.029736   1 ...    0   0   
126  0.210352   0   0   0   0   0  0.056863   0  0.185856   0 ...    0   0   
9    0.985198   1   0   0   0   0  0.096587   0  0.098225   0 ...    0   0   
361  0.500000   1   0   0   0   0  0.017562   0  0.029267   0 ..

In [334]:
# from sklearn.tree import DecisionTreeClassifier

# dt_clf = DecisionTreeClassifier().fit(X_train, y_train)
# dt_predict=dt_clf.predict(X_test)
# dt_accuracy_score=accuracy_score(dt_predict,y_test)
# # scores = cross_val_score(dt_clf, X_train, y_train)
# print(dt_accuracy_score)




'''
neural network classifier
'''
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

n_nodes=20
mlp=MLPClassifier(activation='tanh', hidden_layer_sizes=[n_nodes,n_nodes,n_nodes],random_state=42)
mlp.fit(X_train, y_train)


print("Accuracy of training set: {:.2f}".format(mlp.score(X_train, y_train)))
print("Accuracy of test set: {:.2f}".format(mlp.score(X_test, y_test)))



Accuracy of training set: 0.67
Accuracy of test set: 0.66


In [361]:
y_pred = mlp.predict(df_test)
print(y_pred)

(300, 26)
[1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1]
